In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt
import re

In [2]:
train_df = pd.read_csv("./data/ratings_train.txt", sep = "\t")

In [3]:
test_df = pd.read_csv("./data/ratings_test.txt", sep = "\t")

In [4]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [6]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [7]:
# 결측치 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [8]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [10]:
# 결측치 제거
train_df = train_df.dropna(subset = ["document"])
test_df = test_df.dropna(subset = ["document"])

In [11]:
train_df.shape, test_df.shape

((149995, 3), (49997, 3))

In [12]:
# 종속변수 확인
np.unique(train_df["label"], return_counts = True)

(array([0, 1], dtype=int64), array([75170, 74825], dtype=int64))

# 데이터 전처리

In [13]:
# 한글 이외의 문자들 제거
train_df["document"] = train_df["document"].map(lambda x:re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",x))
test_df["document"] = test_df["document"].map(lambda x:re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",x))

In [14]:
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [15]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


In [16]:
train_df = train_df[train_df["document"].map(lambda x: len(x.strip()) >= 1)]
test_df = test_df[test_df["document"].map(lambda x: len(x.strip()) >= 1)]

In [17]:
train_df.shape, test_df.shape

((148740, 3), (49575, 3))

In [18]:
# 중복 데이터 확인
train_df[train_df["document"].duplicated(keep = False)].sort_values("document")

,id,document,label
46599,9682597,그리고 내 감정을 불러 일으켰다,1
43436,9582856,그리고 내 감정을 불러 일으켰다,1
123713,9582855,그리고 내 감정을 불러 일으켰다,1
93364,171409,가입 추천바람,1
138373,171407,가입 추천바람,1
...,...,...,...
57831,3906478,흥미진진,1
8446,5158304,힐러리 더프의 매력에 빠지다,1
72688,5153363,힐러리 더프의 매력에 빠지다,1
26889,7971814,힘들다,0


In [19]:
# 중복 제거
train_df = train_df.drop_duplicates(subset = ["document"], keep = "first")
test_df = test_df.drop_duplicates(subset = ["document"], keep = "first")

In [20]:
train_df.shape, test_df.shape

((143660, 3), (48403, 3))

# 토큰화

In [21]:
okt = Okt()

In [22]:
okt.morphs("아 더빙 진짜 짜증나네요 목소리", stem = False)

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [23]:
okt.morphs("아 더빙 진짜 짜증나네요 목소리", stem = True)

['아', '더빙', '진짜', '짜증나다', '목소리']

In [24]:
okt.nouns("아 더빙 진짜 짜증나네요 목소리")

['더빙', '진짜', '목소리']

In [25]:
okt.pos("아 더빙 진짜 짜증나네요 목소리")

[('아', 'Exclamation'),
 ('더빙', 'Noun'),
 ('진짜', 'Noun'),
 ('짜증나네요', 'Adjective'),
 ('목소리', 'Noun')]

In [26]:
# 강사님 파일 받아옴
train_df.to_csv("nsmc_ratings_train_pre.csv", index = False)
test_df.to_csv("nsmc_ratings_test_pre.csv", index = False)

In [27]:
# 받아온 파일 읽어서 저장함 
train_df = pd.read_csv("./data/nsmc_ratings_train_pre.csv")
test_df = pd.read_csv("./data/nsmc_ratings_test_pre.csv")

In [28]:
train_df.head()

,id,document,label,token
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"['아', '더빙', '진짜', '짜증나다', '목소리']"
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1,"['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기'..."
2,10265843,너무재밓었다그래서보는것을추천한다,0,"['너', '무재', '밓었', '다그', '래서', '보다', '추천', '한',..."
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0,"['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', ..."
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1,"['사이', '몬페', '그', '의', '익살스럽다', '연기', '가', '돋보..."


In [29]:
test_df.head()

,id,document,label,token
0,6270596,굳 ㅋ,1,"['굳다', 'ㅋ']"
1,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,"['뭐', '야', '이', '평점', '들', '은', '나쁘다', '않다', '..."
2,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,"['지루하다', '않다', '완전', '막장', '임', '돈', '주다', '보기..."
3,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,"['만', '아니다', '별', '다섯', '개', '주다', '왜', '로', '..."
4,7898805,음악이 주가 된 최고의 음악영화,1,"['음악', '이', '주가', '되다', '최고', '의', '음악', '영화']"


In [30]:
type(train_df.loc[0, "token"])

str

In [31]:
train_df.loc[0, "token"]

"['아', '더빙', '진짜', '짜증나다', '목소리']"

In [32]:
train_df["token"] = train_df["token"].map(lambda x: eval(x))
test_df["token"] = test_df["token"].map(lambda x: eval(x))

In [33]:
train_df.shape, test_df.shape

((143660, 4), (48403, 4))

In [34]:
train_df["token"] = train_df["token"].map(lambda x: [i for i in x if len(i) > 1])
test_df["token"] = test_df["token"].map(lambda x: [i for i in x if len(i) > 1])

In [35]:
train_df.head()

,id,document,label,token
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"[더빙, 진짜, 짜증나다, 목소리]"
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1,"[포스터, 보고, 초딩, 영화, 오버, 연기, 조차, 가볍다, 않다]"
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[무재, 밓었, 다그, 래서, 보다, 추천]"
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0,"[교도소, 이야기, 구먼, 솔직하다, 재미, 없다, 평점, 조정]"
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1,"[사이, 몬페, 익살스럽다, 연기, 돋보이다, 영화, 스파이더맨, 에서, 늙다, 보..."


In [36]:
train_df = train_df[train_df["token"].map(lambda x:len(x) > 0)]
test_df = test_df[test_df["token"].map(lambda x:len(x) > 0)]

In [37]:
train_df.shape, test_df.shape

((143319, 4), (48260, 4))

# 정수인코딩 

In [38]:
tokenizer = Tokenizer()

In [39]:
# 단어 집합 생성
# 등장 빈도 수가 높은 순서대로 정수값 부여
tokenizer.fit_on_texts(train_df["token"])

In [40]:
# 단어 집합
tokenizer.word_index

{'영화': 1,
 '보다': 2,
 '하다': 3,
 '없다': 4,
 '이다': 5,
 '있다': 6,
 '좋다': 7,
 '너무': 8,
 '정말': 9,
 '되다': 10,
 '재밌다': 11,
 '같다': 12,
 '진짜': 13,
 '으로': 14,
 '아니다': 15,
 '않다': 16,
 '에서': 17,
 '만들다': 18,
 '나오다': 19,
 '연기': 20,
 '평점': 21,
 '최고': 22,
 '스토리': 23,
 '생각': 24,
 '드라마': 25,
 '사람': 26,
 '감동': 27,
 '보고': 28,
 '이렇다': 29,
 '아깝다': 30,
 '배우': 31,
 'ㅋㅋ': 32,
 '감독': 33,
 '그냥': 34,
 '재미있다': 35,
 '재미': 36,
 '시간': 37,
 '내용': 38,
 '까지': 39,
 '주다': 40,
 '자다': 41,
 '하고': 42,
 '지루하다': 43,
 '재미없다': 44,
 '쓰레기': 45,
 '모르다': 46,
 '가다': 47,
 '들다': 48,
 '그렇다': 49,
 '싶다': 50,
 '작품': 51,
 '사랑': 52,
 '알다': 53,
 '하나': 54,
 '다시': 55,
 '마지막': 56,
 '정도': 57,
 '이건': 58,
 '오다': 59,
 '완전': 60,
 'ㅠㅠ': 61,
 '많다': 62,
 '처음': 63,
 'ㅋㅋㅋ': 64,
 '장면': 65,
 '액션': 66,
 '주인공': 67,
 '이렇게': 68,
 '안되다': 69,
 '차다': 70,
 '나다': 71,
 '이야기': 72,
 '지금': 73,
 '최악': 74,
 '넘다': 75,
 '느낌': 76,
 '연출': 77,
 'ㅡㅡ': 78,
 '인데': 79,
 '좋아하다': 80,
 '명작': 81,
 '그리고': 82,
 '받다': 83,
 '역시': 84,
 '별로': 85,
 '많이': 86,
 '남다': 87,
 '이해': 88,
 '이런': 89,
 '이영

In [41]:
# 단어 등장 수
tokenizer.word_counts

OrderedDict([('더빙', 572),
             ('진짜', 8288),
             ('짜증나다', 1002),
             ('목소리', 374),
             ('포스터', 572),
             ('보고', 4653),
             ('초딩', 422),
             ('영화', 50172),
             ('오버', 142),
             ('연기', 6326),
             ('조차', 242),
             ('가볍다', 360),
             ('않다', 7718),
             ('무재', 69),
             ('밓었', 1),
             ('다그', 75),
             ('래서', 20),
             ('보다', 40991),
             ('추천', 1180),
             ('교도소', 16),
             ('이야기', 2171),
             ('구먼', 11),
             ('솔직하다', 1199),
             ('재미', 3854),
             ('없다', 15519),
             ('평점', 6245),
             ('조정', 40),
             ('사이', 222),
             ('몬페', 2),
             ('익살스럽다', 9),
             ('돋보이다', 288),
             ('스파이더맨', 64),
             ('에서', 6979),
             ('늙다', 198),
             ('보이다', 1073),
             ('하다', 40759),
             ('커스틴', 4),
             (

In [42]:
# 총 단어 등장 수 
total_cnt = len(tokenizer.word_index)
total_cnt

42120

In [111]:
# 사용 단어 수를 지정하여 토큰화
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(train_df["token"])

In [112]:
x_train = tokenizer.texts_to_sequences(train_df["token"])
x_test= tokenizer.texts_to_sequences(test_df["token"])

In [113]:
x_train

[[339, 13, 187, 507],
 [340, 28, 461, 1, 20, 743, 522, 16],
 [2, 152],
 [72, 149, 36, 4, 21],
 [799, 20, 639, 1, 17, 870, 167, 3, 841, 179, 167],
 [98, 155, 1, 64, 857, 30],
 [146, 230, 238, 365, 3],
 [857, 30, 19, 156, 9, 440, 435, 389, 357, 25, 209, 4, 20, 3, 26],
 [66, 4, 36, 6, 69, 1],
 [21, 136, 512, 8, 6],
 [5, 13, 5],
 [568, 285, 968, 139, 276, 462, 620, 5],
 [990, 204, 20, 766, 3],
 [969, 7, 2, 2, 695, 26, 5],
 [680, 13, 239, 17, 2, 1, 124, 258, 27, 23, 27],
 [10, 61],
 [70, 26, 125, 93, 114, 275, 831, 114, 169, 167],
 [927, 88, 3, 384, 36],
 [58, 9, 341, 16, 41],
 [488, 816, 268, 15],
 [249, 6, 34, 1, 268, 15],
 [2, 224, 16],
 [44,
  43,
  12,
  1,
  79,
  42,
  75,
  72,
  6,
  2,
  36,
  6,
  58,
  4,
  85,
  19,
  481,
  147,
  435,
  85,
  19,
  78],
 [268, 609, 1, 15, 294, 205, 956],
 [504, 7, 648, 98, 43],
 [411, 3, 4, 49, 49, 114],
 [94, 3],
 [437],
 [],
 [6, 77, 341, 276, 411],
 [55, 123, 101, 20, 9, 7, 60, 25],
 [45, 375, 175, 32, 30],
 [11, 412, 136],
 [477, 190, 5, 

In [114]:
y_train = np.array(train_df['label'])
y_test = np.array(test_df['label'])

In [115]:
y_train

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [116]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2,
                                                   random_state = 12)

In [117]:
lengths = np.array([len(x) for x in x_train])

In [118]:
# 길이 평균값, 중간값
print(np.mean(lengths), np.median(lengths), np.min(lengths), np.max(lengths))

6.597802101958048 5.0 0 64


In [119]:
train_seq = pad_sequences(x_train, maxlen =10)

In [120]:
print(train_seq.shape)

(114655, 10)


In [121]:
val_seq = pad_sequences(x_val, maxlen = 10)
test_seq = pad_sequences(x_test, maxlen = 10)

In [122]:
# 종속변수는 원핫인코딩
y_oh_train = keras.utils.to_categorical(y_train)
y_oh_val = keras.utils.to_categorical(y_val)
y_oh_test = keras.utils.to_categorical(y_test)

In [127]:
model = keras.Sequential()
model.add(keras.layers.Embedding(1000, 100, input_shape = (10,)))
model.add(keras.layers.Dropout(0.5)) # 밥에 소금, 후추 뿌리는 개념
model.add(keras.layers.Conv1D(64, 5, activation= 'relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool1D(4))
model.add(keras.layers.LSTM(128))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation = 'softmax'))

In [124]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 10, 100)           100000    
                                                                 
 dropout_9 (Dropout)         (None, 10, 100)           0         
                                                                 
 conv1d_4 (Conv1D)           (None, 6, 64)             32064     
                                                                 
 batch_normalization_5 (Bat  (None, 6, 64)             256       
 chNormalization)                                                
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 1, 64)             0         
 g1D)                                                            
                                                                 
 lstm_7 (LSTM)               (None, 128)              

In [128]:
rmsprop = keras.optimizers.RMSprop(learning_rate = 5e-4) 
model.compile(loss='binary_crossentropy', optimizer = rmsprop ,metrics= ['accuracy'])
early_stopping_cb = keras.callbacks.EarlyStopping(patience=4, restore_best_weights= True)

In [129]:
history = model.fit(train_seq, y_oh_train, batch_size = 64, epochs = 100,
                   validation_data=(val_seq, y_oh_val),
                   callbacks= [early_stopping_cb])

Epoch 1/100
1792/1792 [==============================] - 13s 6ms/step - loss: 0.5863 - accuracy: 0.6643 - val_loss: 0.5458 - val_accuracy: 0.6995
Epoch 2/100
1792/1792 [==============================] - 10s 6ms/step - loss: 0.5480 - accuracy: 0.7004 - val_loss: 0.5397 - val_accuracy: 0.7037
Epoch 3/100
1792/1792 [==============================] - 9s 5ms/step - loss: 0.5406 - accuracy: 0.7062 - val_loss: 0.5371 - val_accuracy: 0.7058
Epoch 4/100
1792/1792 [==============================] - 10s 6ms/step - loss: 0.5367 - accuracy: 0.7083 - val_loss: 0.5387 - val_accuracy: 0.7051
Epoch 5/100
1792/1792 [==============================] - 10s 5ms/step - loss: 0.5337 - accuracy: 0.7111 - val_loss: 0.5351 - val_accuracy: 0.7078
Epoch 6/100
1792/1792 [==============================] - 10s 5ms/step - loss: 0.5308 - accuracy: 0.7128 - val_loss: 0.5342 - val_accuracy: 0.7076
Epoch 7/100
1792/1792 [==============================] - 10s 6ms/step - loss: 0.5272 - accuracy: 0.7150 - val_loss: 0.5355 - 